---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 1 - Creating and Manipulating Graphs

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [4]:
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
def plot_graph(G, weight_name=None):
    '''
    G: a networkx G
    weight_name: name of the attribute for plotting edge weights (if G is weighted)
    '''
    %matplotlib notebook
    import matplotlib.pyplot as plt
    
    plt.figure()
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, edges=edges, width=weights);
    else:
        nx.draw_networkx(G, pos, edges=edges);

### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

*This function should return a networkx graph with 19 nodes and 24 edges*

In [5]:
def answer_one():
        
    # Your Code Here
    G = nx.read_edgelist('Employee_Movie_Choices.txt', comments = '#', delimiter = '\t')
    
    return G
#plot_graph(answer_one())

### Question 2

Using the graph from the previous question, add nodes attributes named `'type'` where movies have the value `'movie'` and employees have the value `'employee'` and return that graph.

*This function should return a networkx graph with node attributes `{'type': 'movie'}` or `{'type': 'employee'}`*

In [6]:
def answer_two():
    
    # Your Code Here
    G = answer_one()
    for n in list(G.nodes(data=False)):
        G.node[n]['type'] = 'employee' if n in employees else 'movie'
    
    return G
#answer_two().nodes(data=True)

### Question 3

Find a weighted projection of the graph from `answer_two` which tells us how many movies different pairs of employees have in common.

*This function should return a weighted projected graph.*

In [7]:
def answer_three():
        
    # Your Code Here
    G = answer_two()
    WLG = bipartite.weighted_projected_graph(G, employees)
    
    return WLG
#plot_graph(answer_three())

### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [8]:
def answer_four():
        
    # Your Code Here
    movie_choices = [list(elem) for elem in list(answer_three().edges.data('weight'))]
    # movie_choices = [list(elem) for elem in list(answer_three().edges_iter(data = 'weight'))] # versiom 1.11
    for s in movie_choices:
        for i in range(min(len(s[0]), len(s[1]))):
            if s[0][i] != s[1][i]:
                c = i
                break
        if s[0][c] > s[1][c]:
            tmp = s[0]
            s[0] = s[1]
            s[1] = tmp
    ms_df = pd.DataFrame(movie_choices)\
            .rename(columns = {0:'person1', 1:'person2', 2:'Number of same movies'})\
            .set_index(['person1', 'person2'])\
            .sort_index()
    rs_df = pd.read_csv('Employee_Relationships.txt', sep='\t', header=None)\
            .rename(columns = {0:'person1', 1:'person2', 2:'Relationships'})\
            .set_index(['person1', 'person2'])\
            .sort_index()
    df = ms_df.merge(rs_df, how = 'outer', left_index=True, right_index=True).fillna(0)
    return df#.iloc[:, 0].corr(df.iloc[:, 1])
answer_four()

Number of same movies  Relationships
person1 person2                                      
Andy    Claude                     1.0              0
        Frida                      1.0             20
        Georgia                    1.0            -10
        Joan                       1.0             30
        Lee                        1.0            -10
        Pablo                      1.0            -10
        Vincent                    0.0             20
Claude  Frida                      0.0              0
        Georgia                    3.0             90
        Joan                       0.0              0
        Lee                        0.0              0
        Pablo                      0.0             10
        Vincent                    0.0              0
Frida   Georgia                    0.0              0
        Joan                       0.0              0
        Lee                        0.0              0
        Pablo                      2.0             50
        Vincent                    2.0             60
Georgia Joan                       0.0              0
        Lee                        0.0             10
        Pablo                      0.0              0
        Vincent                    0.0              0
Joan    Lee                        3.0             70
        Pablo                      0.0              0
        Vincent                    0.0             10
Lee     Pablo                      0.0              0
        Vincent                    0.0              0
Pablo   Vincent                    1.0            -20